# Employees Data with Delta Lake (Python)


## Configuration

In [1]:
import random

delta_table_path = "/delta/delta-table-customer"
delta_table_path

StatementMeta(, , , SessionError, )

LivyHttpRequestFailure: Something went wrong while processing your request. Please try again later. HTTP status code: 500. Trace ID: 592663dc-786f-4049-8a90-f2b0755800b7.

## Create a dataframe

In [22]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data2 = [("Raul","","Sarachaga","36636","M",3000),
    ("Alejandro","Sanchez","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Martha","Chavez","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)
df.printSchema()
df.show(truncate=False)
df.write.format("delta").save(delta_table_path)

StatementMeta(SparkPoll, 0, 21, Finished, Available)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+---------+-----+------+------+
|firstname|middlename|lastname |id   |gender|salary|
+---------+----------+---------+-----+------+------+
|Raul     |          |Sarachaga|36636|M     |3000  |
|Alejandro|Sanchez   |         |40288|M     |4000  |
|Robert   |          |Williams |42114|M     |4000  |
|Martha   |Chavez    |Jones    |39192|F     |4000  |
|Jen      |Mary      |Brown    |     |F     |-1    |
+---------+----------+---------+-----+------+------+

## Understanding Meta-data

In [23]:
[log_line.value for log_line in spark.read.text(delta_table_path + "/_delta_log/").collect()]

StatementMeta(SparkPoll, 0, 22, Finished, Available)

['{"commitInfo":{"timestamp":1619650132469,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"},"isBlindAppend":true,"operationMetrics":{"numFiles":"6","numOutputBytes":"8126","numOutputRows":"5"}}}', '{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}', '{"metaData":{"id":"8384e9fa-e807-45bc-bc2b-3a2d1d6dee7e","format":{"provider":"parquet","options":{}},"schemaString":"{\\"type\\":\\"struct\\",\\"fields\\":[{\\"name\\":\\"firstname\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"middlename\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"lastname\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"id\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"gender\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"salary\\",\\"type\\":\\"integer\\",\\"nullable\\":true,\\"metadata\\":{}}]}","partitionC

## Read data

In [24]:
df = spark.read.format("delta").load(delta_table_path)
df.show()

StatementMeta(SparkPoll, 0, 23, Finished, Available)

+---------+----------+---------+-----+------+------+
|firstname|middlename| lastname|   id|gender|salary|
+---------+----------+---------+-----+------+------+
|   Martha|    Chavez|    Jones|39192|     F|  4000|
|Alejandro|   Sanchez|         |40288|     M|  4000|
|   Robert|          | Williams|42114|     M|  4000|
|     Raul|          |Sarachaga|36636|     M|  3000|
|      Jen|      Mary|    Brown|     |     F|    -1|
+---------+----------+---------+-----+------+------+

## Update data

In [26]:
data2 = [("Raul","","Sarachaga","36636","M",3000),
    ("Alejandro","Sanchez","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Martha","Chavez","Jones","39192","F",4000)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])

data = spark.createDataFrame(data=data2,schema=schema)
data.write.format("delta").mode("overwrite").save(delta_table_path)
df.show()

StatementMeta(SparkPoll, 0, 25, Finished, Available)

+---------+----------+---------+-----+------+------+
|firstname|middlename| lastname|   id|gender|salary|
+---------+----------+---------+-----+------+------+
|   Martha|    Chavez|    Jones|39192|     F|  4000|
|Alejandro|   Sanchez|         |40288|     M|  4000|
|   Robert|          | Williams|42114|     M|  4000|
|     Raul|          |Sarachaga|36636|     M|  3000|
+---------+----------+---------+-----+------+------+

In [27]:
[log_line.value for log_line in spark.read.text(delta_table_path + "/_delta_log/").collect()]

StatementMeta(SparkPoll, 0, 26, Finished, Available)

['{"commitInfo":{"timestamp":1619650132469,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"},"isBlindAppend":true,"operationMetrics":{"numFiles":"6","numOutputBytes":"8126","numOutputRows":"5"}}}', '{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}', '{"metaData":{"id":"8384e9fa-e807-45bc-bc2b-3a2d1d6dee7e","format":{"provider":"parquet","options":{}},"schemaString":"{\\"type\\":\\"struct\\",\\"fields\\":[{\\"name\\":\\"firstname\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"middlename\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"lastname\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"id\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"gender\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"salary\\",\\"type\\":\\"integer\\",\\"nullable\\":true,\\"metadata\\":{}}]}","partitionC

## Save as tables

In [29]:
# Define an external catalog table that points to the existing Delta Lake data in storage.
spark.sql("CREATE TABLE CustomerDeltaTable USING DELTA LOCATION '{0}'".format(delta_table_path))

StatementMeta(SparkPoll, 0, 28, Finished, Available)

DataFrame[]

In [30]:
%%sql
SELECT * FROM CustomerDeltaTable

StatementMeta(SparkPoll, 0, 29, Finished, Available)

<Spark SQL result set with 4 rows and 6 fields>

## Update rows

In [31]:
from delta.tables import *
from pyspark.sql.functions import *

delta_table = DeltaTable.forPath(spark, delta_table_path)

StatementMeta(SparkPoll, 0, 30, Finished, Available)

In [34]:
# 
delta_table.update(
  condition = expr("id  == 39192"),
  set = { "salary": "1000" })
delta_table.toDF().show()

StatementMeta(SparkPoll, 0, 33, Finished, Available)

+---------+----------+---------+-----+------+------+
|firstname|middlename| lastname|   id|gender|salary|
+---------+----------+---------+-----+------+------+
|   Martha|    Chavez|    Jones|39192|     F|  1000|
|Alejandro|   Sanchez|         |40288|     M|  4000|
|   Robert|          | Williams|42114|     M|  4000|
|     Raul|          |Sarachaga|36636|     M|  3000|
+---------+----------+---------+-----+------+------+

## Delete rows

In [35]:
# Delete every even value
delta_table.delete("id == 40288")
delta_table.toDF().show()

StatementMeta(SparkPoll, 0, 34, Finished, Available)

+---------+----------+---------+-----+------+------+
|firstname|middlename| lastname|   id|gender|salary|
+---------+----------+---------+-----+------+------+
|   Martha|    Chavez|    Jones|39192|     F|  1000|
|   Robert|          | Williams|42114|     M|  4000|
|     Raul|          |Sarachaga|36636|     M|  3000|
+---------+----------+---------+-----+------+------+

In [38]:
data2 = [("Martin","","Sarachaga","36637","M",3000)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])

new_data  = spark.createDataFrame(data=data2,schema=schema)
new_data .show()

StatementMeta(SparkPoll, 0, 37, Finished, Available)

+---------+----------+---------+-----+------+------+
|firstname|middlename| lastname|   id|gender|salary|
+---------+----------+---------+-----+------+------+
|   Martin|          |Sarachaga|36637|     M|  3000|
+---------+----------+---------+-----+------+------+

## Merge Rows

In [42]:
# Upsert (merge) new data
#new_data = spark.range(0,20).alias("newData")

delta_table.alias("oldData")\
    .merge(new_data.alias("newData"), "oldData.id = newData.id")\
    .whenMatchedUpdate(set = { "id": lit("-1")})\
    .whenNotMatchedInsert(values = { "id": col("newData.id") , 
                                    "firstname": col("newData.firstname"), 
                                    "lastname": col("newData.lastname"),
                                    "gender": col("newData.gender"),
                                    "salary": col("newData.salary") })\
    .execute()

delta_table.toDF().show(100)

StatementMeta(SparkPoll, 0, 41, Finished, Available)

+---------+----------+---------+-----+------+------+
|firstname|middlename| lastname|   id|gender|salary|
+---------+----------+---------+-----+------+------+
|   Martha|    Chavez|    Jones|39192|     F|  1000|
|   Robert|          | Williams|42114|     M|  4000|
|     Raul|          |Sarachaga|36636|     M|  3000|
|   Martin|      null|Sarachaga|   -1|  null|  null|
|     null|      null|     null|   -1|  null|  null|
+---------+----------+---------+-----+------+------+

## History

In [47]:
delta_table.history().show(20,100)

StatementMeta(SparkPoll, 0, 46, Finished, Available)

+-------+-------------------+------+--------+---------+----------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------+
|version|          timestamp|userId|userName|operation|                           operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|                                                                                    operationMetrics|
+-------+-------------------+------+--------+---------+----------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------+
|      8|2021-04-28 23:01:55|  null|    null|    MERGE|  [predicate -> (oldData.`id` = newData.`id`)]|null|    null|     null|          7|          null|        false|[numTargetRowsCopied -> 0,